When you’re defining a module’s API, the exceptions you throw are just as much a part of
your interface as the functions and classes you define (see Item 14: “Prefer Exceptions to
Returning None”).

Python has a built-in hierarchy of exceptions for the language and standard library.
There’s a draw to using the built-in exception types for reporting errors instead of defining
your own new types. For example, you could raise a ValueError exception whenever
an invalid parameter is passed to your function.


In [1]:
import logging
from pprint import pprint
from sys import stdout as STDOUT


# Example 1
try:
    def determine_weight(volume, density):
        if density <= 0:
            raise ValueError('Density must be positive')
    
    determine_weight(1, 0)
except:
    logging.exception('Expected')
else:
    assert False


ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-1-586bac59af4c>", line 12, in <module>
    determine_weight(1, 0)
  File "<ipython-input-1-586bac59af4c>", line 10, in determine_weight
    raise ValueError('Density must be positive')
ValueError: Density must be positive


### Ex 2
In some cases, using ValueError makes sense, but for APIs it’s much more powerful to
define your own hierarchy of exceptions. You can do this by providing a root
Exception in your module. Then, have all other exceptions raised by that module
inherit from the root exception.

### Ex 3
Having a root exception in a module makes it easy for consumers of your API to catch all
of the exceptions that you raise on purpose. For example, here a consumer of your API

This try/except prevents your API’s exceptions from propagating too far upward and
breaking the calling program. It insulates the calling code from your API. This insulation
has three helpful effects.

### Ex 4
First, root exceptions let callers understand when there’s a problem with their usage of
your API. If callers are using your API properly, they should catch the various exceptions
that you deliberately raise. If they don’t handle such an exception, it will propagate all the
way up to the insulating except block that catches your module’s root exception. That
block can bring the exception to the attention of the API consumer, giving them a chance
to add proper handling of the exception type.

### Ex 5
The second advantage of using root exceptions is that they can help find bugs in your API
module’s code. If your code only deliberately raises exceptions that you define within
your module’s hierarchy, then all other types of exceptions raised by your module must be
the ones that you didn’t intend to raise. These are bugs in your API’s code.
Using the try/except statement above will not insulate API consumers from bugs in
your API module’s code. To do that, the caller needs to add another except block that
catches Python’s base Exception class. This allows the API consumer to detect when
there’s a bug in the API module’s implementation that needs to be fixed.

### Ex 6
The third impact of using root exceptions is future-proofing your API. Over time, you may
want to expand your API to provide more specific exceptions in certain situations. For
example, you could add an Exception subclass that indicates the error condition of
supplying negative densities.

In [3]:
# Example 2
# my_module.py
class Error(Exception):
    """Base-class for all exceptions raised by this module."""

class InvalidDensityError(Error):
    """There was a problem with a provided density value."""
    
# Example 3
class my_module(object):
    Error = Error
    InvalidDensityError = InvalidDensityError

    @staticmethod
    def determine_weight(volume, density):
        if density <= 0:
            raise InvalidDensityError('Density must be positive')
try:
    weight = my_module.determine_weight(1, -1)
    assert False
except my_module.Error as e:
    logging.error('Unexpected error: %s', e)


# Example 4
weight = 5
try:
    weight = my_module.determine_weight(1, -1)
    assert False
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error('Bug in the calling code: %s', e)

assert weight == 0


# Example 5
weight = 5
try:
    weight = my_module.determine_weight(1, -1)
    assert False
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error('Bug in the calling code: %s', e)
except Exception as e:
    logging.error('Bug in the API code: %s', e)
    raise

assert weight == 0


# Example 6
# my_module.py
class NegativeDensityError(InvalidDensityError):
    """A provided density value was negative."""

def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError




ERROR:root:Unexpected error: Density must be positive


### Ex 7
The calling code will continue to work exactly as before because it already catches
InvalidDensityError exceptions (the parent class of
NegativeDensityError). In the future, the caller could decide to special-case the
new type of exception and change its behavior accordingly.

### Ex 8
You can take API future-proofing further by providing a broader set of exceptions directly
below the root exception. For example, imagine you had one set of errors related to
calculating weights, another related to calculating volume, and a third related to
calculating density.


Specific exceptions would inherit from these general exceptions. Each intermediate
exception acts as its own kind of root exception. This makes it easier to insulate layers of
calling code from API code based on broad functionality. This is much better than having
all callers catch a long list of very specific Exception subclasses.


In [4]:
# Example 7
try:
    my_module.NegativeDensityError = NegativeDensityError
    my_module.determine_weight = determine_weight
    try:
        weight = my_module.determine_weight(1, -1)
        assert False
    except my_module.NegativeDensityError as e:
        raise ValueError('Must supply non-negative density') from e
    except my_module.InvalidDensityError:
        weight = 0
    except my_module.Error as e:
        logging.error('Bug in the calling code: %s', e)
    except Exception as e:
        logging.error('Bug in the API code: %s', e)
        raise
except:
    logging.exception('Expected')
else:
    assert False


# Example 8
# my_module.py
class WeightError(Error):
    """Base-class for weight calculation errors."""

class VolumeError(Error):
    """Base-class for volume calculation errors."""

class DensityError(Error):
    """Base-class for density calculation errors."""

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-4-7c217a018815>", line 3, in <module>
    my_module.NegativeDensityError = NegativeDensityError
NameError: name 'NegativeDensityError' is not defined


* Defining root exceptions for your modules allows API consumers to insulate themselves from your API.
* Catching root exceptions can help you find bugs in code that consumes an API. 
* Catching the Python Exception base class can help you find bugs in API implementations.
* Intermediate root exceptions let you add more specific types of exceptions in the future without breaking your API consumers.
